In [1]:
# Aloe Token
token="74b1aaf8ca3f131071be811964fd2987"

# Abaco Tools

In [2]:
import os
import time 
import numpy as np
import pandas as pd
import pprint as pp
import requests as r

header_dat={'Authorization': f'Bearer {token}'}
url = "https://dev.tenants.aloedev.tacc.cloud/actors/v2"

In [9]:
# Get all actors.
actor = r.get(f"{url}/YJE3V4qQ5rA5/executions",
               headers=header_dat)
actor.json()

{'message': 'Actor executions retrieved successfully.',
 'result': {'_links': {'owner': 'http://172.17.0.1:8000/profiles/v2/testuser8',
   'self': 'http://172.17.0.1:8000/actors/v2/YJE3V4qQ5rA5/executions'},
  'actorId': 'YJE3V4qQ5rA5',
  'apiServer': 'http://172.17.0.1:8000',
  'executions': [{'id': 'RrbObDmVxgwXq',
    'message_received_time': '2019-06-13 23:01:06.248910',
    'start_time': None,
    'status': 'SUBMITTED'},
   {'id': 'LlwJNBV6gBGPW',
    'message_received_time': '2019-06-13 23:01:06.341119',
    'start_time': None,
    'status': 'SUBMITTED'},
   {'id': '0GeBag0yWDjGz',
    'message_received_time': '2019-06-13 23:01:06.344791',
    'start_time': None,
    'status': 'SUBMITTED'},
   {'id': 'PlEQxJRpG0OAl',
    'message_received_time': '2019-06-13 23:01:06.423031',
    'start_time': None,
    'status': 'SUBMITTED'},
   {'id': 'qwqNgXjZEyJap',
    'message_received_time': '2019-06-13 23:01:06.427005',
    'start_time': None,
    'status': 'SUBMITTED'},
   {'id': 'pBZQX44

In [8]:
# Get all actors.
actor = r.get(f"{url}",
               headers=header_dat)
actor.json()

{'message': 'Actors retrieved successfully.',
 'result': [{'_links': {'executions': 'http://172.17.0.1:8000/actors/v2/YJE3V4qQ5rA5/executions',
    'owner': 'http://172.17.0.1:8000/profiles/v2/testuser8',
    'self': 'http://172.17.0.1:8000/actors/v2/YJE3V4qQ5rA5'},
   'createTime': '2019-06-13 23:00:37.909498',
   'defaultEnvironment': {},
   'description': '',
   'id': 'YJE3V4qQ5rA5',
   'image': 'notchristiangarcia/flops_test:test-1.0',
   'lastUpdateTime': '2019-06-13 23:00:37.909498',
   'mounts': [{'container_path': '/_abaco_data1',
     'host_path': '/home/apim/data1',
     'mode': 'ro'},
    {'container_path': '/_abaco_data2',
     'host_path': '/home/apim/data2',
     'mode': 'rw'}],
   'owner': 'testuser8',
   'privileged': False,
   'queue': 'default',
   'state': {},
   'stateless': True,
   'status': 'SUBMITTED',
   'statusMessage': '',
   'type': 'none',
   'useContainerUid': False}],
 'status': 'success',
 'version': ':dev'}

In [7]:
# Create actor for work
actor = r.post(f"{url}",
               headers=header_dat,
               data={'image':'notchristiangarcia/flops_test:test-1.0'})
actor_id = actor.json()['result']['id']
actor.json();

In [5]:
# Delete specified actors.
for del_actor in ['qAMA8MLmwGkb3' ,'RrOGMZpo5Az5o']:
    del_actor_res = r.delete(f"{url}/{del_actor}",
                     headers=header_dat)
    print(del_actor_res.json())

{'message': 'Actor deleted successfully.', 'result': None, 'status': 'success', 'version': ':dev'}
{'message': 'Actor deleted successfully.', 'result': None, 'status': 'success', 'version': ':dev'}


In [4]:
# Check number of workers currently available
worker_res = r.get(f"{url}/{actor_id}/workers",
                    headers=header_dat)
num_workers = len(worker_res.json()['result'])
print(num_workers)

host_info = {}
all_workers = []
try:
    if num_workers:
        for worker_info in worker_res.json()['result']:
            if f"Host {worker_info['hostId']}" in host_info:
                host_info[f"Host {worker_info['hostId']}"] += 1
            else:
                host_info[f"Host {worker_info['hostId']}"] = 1
        for worker_info in worker_res.json()['result']:
            all_workers.append(worker_info['id'])
except KeyError:
    print('Still spooling.')
worker_res.json();
host_info

NameError: name 'actor_id' is not defined

In [65]:
# Spool up more workers
worker_res = r.post(f"{url}/{actor_id}/workers",
                    headers=header_dat,
                    data={'num': 3})
worker_res.json()

{'message': 'Scheduled 3 new worker(s) to start. Previously, there were 0 workers.',
 'result': None,
 'status': 'success',
 'version': ':dev'}

In [64]:
# Delete all current workers!
worker_res = r.get(f"{url}/{actor_id}/workers",
                        headers=header_dat)
print('Deleted Workers:')
for worker_info in worker_res.json()['result']:
    del_worker_res = r.delete(f"{url}/{actor_id}/workers/{worker_info['id']}",
                        headers=header_dat)
    print('.', end='')

Deleted Workers:
.

# Everything that you really just don't need to see at all.

In [32]:
nodes = 2 #os.environ['nodes']

# Get all current workers
worker_res = r.get(f"{url}/{actor_id}/workers",
                        headers=header_dat)
all_workers = []
for worker_info in worker_res.json()['result']:
    all_workers.append(worker_info['id'])
    
# Delete all current workers
for del_worker in all_workers:
    del_worker_res = r.delete(f"{url}/{actor_id}/workers/{del_worker}",
                        headers=header_dat)

# Spool up more workers!
worker_res = r.post(f"{url}/{actor_id}/workers",
                    headers=header_dat,
                    data={'num': nodes*6})

# Check on amount of workers
while True:
    worker_res = r.get(f"{url}/{actor_id}/workers",
                        headers=header_dat)
    num_workers = len(worker_res.json()['result'])
    if not num_workers == nodes*6:
        continue
    
    host_info = {}
    if num_workers:
        try:
            for worker_info in worker_res.json()['result']:
                worker_info['hostId']
            break
        except KeyError:
            continue

In [ ]:
all_data = pd.DataFrame()

actor_id = 'peorL6Eg4GPW3'

nodes = 1
workers_per_node = 6
messages_per_worker = 10

threads = 0        # set to 0 for all threads
std_dev = 1000
size = 1000
executions = nodes * workers_per_node * messages_per_worker
messageDat = f'{threads} {std_dev} {size}'
runs = 3

for run_num in range(1, runs + 1):
    print(f'Starting Run {run_num}.')
    msg_start = time.time()
    exec_id_list = []
    for i in range(executions):
        execution = r.post(f"{url}/{actor_id}/messages",
                           headers=header_dat,
                           data={'message':messageDat})
        exec_id = execution.json()['result']['executionId']
        exec_id_list.append(exec_id)
        print(f'\rCreated execution: {i+1}', end='')
    msg_end = time.time()

    exec_start = time.time()
    results_list = []
    
    executions_completed = 0
    while exec_id_list:
        for exec_id in exec_id_list:
            exec_logs = r.get(f"{url}/{actor_id}/executions/{exec_id}/logs",
                              headers=header_dat)

            logs = exec_logs.json()['result']['logs']
            if logs:
                print(f'\rExecutions completed: {executions_completed + 1}', end='')
                results_list.append([f'id - {exec_id}', float(logs.replace('\n',''))])
                exec_id_list.remove(exec_id)
                executions_completed += 1
    exec_end = time.time()

    work_time = 0
    for res in results_list:
        work_time += res[1]

    print(f"\n\nRun Number {run_num}")
    if threads:
        print(f"Threads: {threads}")
    print(f"Std Dev: {std_dev}")
    print(f"Size: {size}")
    print(f"Executions: {executions}")
    print(f"Message Time: {msg_end - msg_start}")
    print(f"Exec Time: {exec_end - exec_start}")
    print(f"Work Time: {work_time}")
    print('\n')

    run_data = pd.DataFrame([[run_num, threads, std_dev, size, executions, msg_end - msg_start, exec_end - exec_start, work_time]],
                            columns=['Run Number', 'Threads', 'Std Dev', 'Size', 'Executions', 'Message Time', 'Exec Time', 'Work Time'])
    all_data = all_data.append(run_data, ignore_index = True)
all_data.to_csv(f'{nodes}_nodes_{runs}_trials.csv')

# Hyper Parameter Testing

In [ ]:
# Hyper paramater tester
actor_id = 'peorL6Eg4GPW3'

all_data = pd.DataFrame()
for test_var in [1000, 2000, 3000]:
    nodes = 1
    workers_per_node = 6
    messages_per_worker = 5

    threads = 0        # set to 0 for all threads
    std_dev = 1000
    size = test_var
    executions = nodes * workers_per_node * messages_per_worker
    messageDat = f'{threads} {std_dev} {size}'
    runs = 3

    for run_num in range(1, runs + 1):
        print(f'Starting Run {run_num}.')
        msg_start = time.time()
        exec_id_list = []
        for i in range(executions):
            execution = r.post(f"{url}/{actor_id}/messages",
                               headers=header_dat,
                               data={'message':messageDat})
            exec_id = execution.json()['result']['executionId']
            exec_id_list.append(exec_id)
            print(f'\rCreated execution: {i+1}', end='')
        msg_end = time.time()

        exec_start = time.time()
        results_list = []

        executions_completed = 0
        while exec_id_list:
            for exec_id in exec_id_list:
                exec_logs = r.get(f"{url}/{actor_id}/executions/{exec_id}/logs",
                                  headers=header_dat)

                logs = exec_logs.json()['result']['logs']
                if logs:
                    print(f'\rExecutions completed: {executions_completed + 1}', end='')
                    results_list.append([f'id - {exec_id}', float(logs.replace('\n',''))])
                    exec_id_list.remove(exec_id)
                    executions_completed += 1
        exec_end = time.time()

        work_time = 0
        for res in results_list:
            work_time += res[1]

        print(f"\n\nRun Number {run_num}")
        if threads:
            print(f"Threads: {threads}")
        print(f"Std Dev: {std_dev}")
        print(f"Size: {size}")
        print(f"Executions: {executions}")
        print(f"Message Time: {msg_end - msg_start}")
        print(f"Exec Time: {exec_end - exec_start}")
        print(f"Work Time: {work_time}")
        print('\n')

    run_data = pd.DataFrame([[run_num, threads, std_dev, size, executions, msg_end - msg_start, exec_end - exec_start, work_time]],
                            columns=['Run Number', 'Threads', 'Std Dev', 'Size', 'Executions', 'Message Time', 'Exec Time', 'Work Time'])
    all_data = all_data.append(run_data, ignore_index = True)
all_data.to_csv(f'{nodes}_nodes_{runs}_trials.csv')


# LEGACY

In [ ]:
# message = "'std_deviation' 'size'"
messageDat = '3 10 1000'
executions = 8

# Actor that will be creating workers with flop test.
actor = r.post(f"{url}/actors",
               headers=header_dat,
               data={'image':'notchristiangarcia/flops_test'})
actor_id = actor.json()['result']['id']


# Executions of this actor, lots of times.
exec_id_list = []
for i in range(executions):
    execution = r.post(f"{url}/actors/{actor_id}/messages",
                       headers=header_dat,
                       data={'message':messageDat})
    exec_id = execution.json()['result']['executionId']
    exec_id_list.append(exec_id)


# Getting result of exec from log once exec complete.
results_dict = {}
while exec_id_list:
    for exec_id in exec_id_list:
        exec_logs = r.get(f"{url}/actors/{actor_id}/executions/{exec_id}/logs",
                          headers=header_dat)

        logs = exec_logs.json()['result']['logs']
        if logs:
            results_dict[f'id - {exec_id}'] = float(logs.replace('\n',''))
            exec_id_list.remove(exec_id)

In [ ]:
# message = "'threads' 'std_deviation' 'size'"
messageDat = '1 10 1500'
executions = 8

msg_start = time.time()
exec_id_list = []
for i in range(executions):
    execution = r.post(f"{url}/actors/{actor_id}/messages",
                       headers=header_dat,
                       data={'message':messageDat})
    exec_id = execution.json()['result']['executionId']
    exec_id_list.append(exec_id)
msg_end = time.time()

In [ ]:
exec_start = time.time()
results_list = []
while exec_id_list:
    for exec_id in exec_id_list:
        exec_logs = r.get(f"{url}/actors/{actor_id}/executions/{exec_id}/logs",
                          headers=header_dat)

        logs = exec_logs.json()['result']['logs']
        if logs:
            print('. ', end='')
            results_list.append([f'id - {exec_id}', float(logs.replace('\n',''))])
            exec_id_list.remove(exec_id)
exec_end = time.time()

work_time = 0
for res in results_list:
    work_time += res[1]

print(f"\n\nExec Time: {exec_end - exec_start}")
print(f"Message Time: {msg_end - msg_start}")
print(f"Work Time: {work_time}")
pp.pprint(results_list)